In [ ]:
print('Test')

In [ ]:
import os
import json
import gc
from pathlib import Path

In [ ]:
# CONFIG
CARPETA_TXT = "Convert_Books"
CARPETA_CHUNKS = "Chunks_Dataset"
CHUNK_SIZE = 1800
OVERLAP = 300
BLOCK_SIZE = 1_000_000  # leer 1 MB a la vez; ajústalo según RAM

def dividir_en_chunks_from_file(path, size=CHUNK_SIZE, overlap=OVERLAP, block_size=BLOCK_SIZE):
    """
    Generador que lee el archivo por bloques y produce (index, chunk) sin cargar todo el archivo.
    """
    buffer = ""
    idx = 0
    with open(path, "r", encoding="utf-8") as f:
        while True:
            parte = f.read(block_size)
            if not parte:
                break
            buffer += parte

            # mientras haya suficiente texto en buffer, extraer chunks
            while len(buffer) >= size:
                ventana = buffer[:size]

                corte = max(
                    ventana.rfind("."),
                    ventana.rfind("?"),
                    ventana.rfind("!"),
                    ventana.rfind("\n")
                )

                if corte != -1 and corte > size * 0.4:
                    len_chunk = corte + 1
                else:
                    len_chunk = size

                chunk = buffer[:len_chunk].strip()
                if chunk:
                    yield idx, chunk
                    idx += 1

                # mantener solapamiento en buffer
                buffer = buffer[len_chunk - overlap:]

    # procesar resto del buffer final
    # podemos seguir cortando hasta que quede muy pequeño
    while buffer:
        if len(buffer) <= 0:
            break
        ventana = buffer
        corte = max(
            ventana.rfind("."),
            ventana.rfind("?"),
            ventana.rfind("!"),
            ventana.rfind("\n")
        )
        if corte != -1 and corte > len(buffer) * 0.4:
            len_chunk = corte + 1
        else:
            # si es pequeño, tomar todo
            len_chunk = len(buffer)

        chunk = buffer[:len_chunk].strip()
        if chunk:
            yield idx, chunk
            idx += 1

        buffer = buffer[len_chunk - overlap:]

In [ ]:
def generar_chunks_stream():
    carpeta_origen = Path(CARPETA_TXT)
    carpeta_destino = Path(CARPETA_CHUNKS)
    carpeta_destino.mkdir(parents=True, exist_ok=True)

    for archivo in carpeta_origen.glob("*.txt"):
        print("Procesando:", archivo.name)
        carpeta_libro = carpeta_destino / archivo.stem
        carpeta_libro.mkdir(exist_ok=True)

        total = 0
        for i, chunk in dividir_en_chunks_from_file(archivo):
            data = {
                "id": f"{archivo.stem}_chunk_{i}",
                "book_id": archivo.stem,
                "text": chunk,
                "chunk_index": i,
                "char_count": len(chunk),
                "word_count": len(chunk.split())
            }

            ruta = carpeta_libro / f"{archivo.stem}_chunk_{i}.json"
            with open(ruta, "w", encoding="utf-8") as out_f:
                json.dump(data, out_f, indent=2, ensure_ascii=False)

            total += 1
            if total % 100 == 0:
                # mensaje de progreso discreto
                print(f"  → {total} chunks escritos...")

        print(f"✓ {total} chunks generados para {archivo.name}\n")

        # liberar memoria
        gc.collect()

In [ ]:
# Antes de ejecutar, asegúrate de haber cerrado salidas previas y reiniciado kernel si hubo OOM.
generar_chunks_stream()

In [ ]:
from datasets import Dataset
import json
from pathlib import Path

# Ruta de la carpeta donde guardaste los chunks
carpeta = Path("Chunks_Dataset")

# Lista para almacenar todos los datos
registros = []

# Cargar todos los archivos JSON
for archivo_json in carpeta.rglob("*.json"):
    with open(archivo_json, "r", encoding="utf-8") as f:
        data = json.load(f)
        registros.append(data)

# Verificar cuántos registros hemos cargado
len(registros), registros[0]


In [ ]:
dataset = Dataset.from_list(registros)
dataset


In [ ]:
from transformers import GPT2Tokenizer

tokenizer = GPT2Tokenizer.from_pretrained("gpt2")

def tokenizar(batch):
    return tokenizer(
        batch["text"],
        truncation=True,
        padding="max_length",
        max_length=512
    )

# Tokenizar todos los registros
tokenized = dataset.map(tokenizar, batched=True)

# Eliminar columnas innecesarias
tokenized = tokenized.remove_columns([col for col in tokenized.column_names if col != "input_ids"])
tokenized.set_format("torch")

tokenized

In [ ]:
from transformers import GPT2LMHeadModel

model = GPT2LMHeadModel.from_pretrained("gpt2")


In [ ]:
for name, param in model.named_parameters():
    if "h." in name:
        capa = int(name.split(".")[1])
        if capa < 6:   # Congelamos capas 0–5
            param.requires_grad = False

print("Capas 0–5 congeladas. Solo las capas 6–11 serán entrenadas.")

In [ ]:
from transformers import TrainingArguments

args = TrainingArguments(
    output_dir="modelo_final",          # carpeta donde guardaremos el modelo entrenado
    evaluation_strategy="steps",
    eval_steps=200,
    logging_steps=50,
    learning_rate=5e-5,
    weight_decay=0.01,
    num_train_epochs=1,                 # Puedes aumentar las épocas si lo deseas
    per_device_train_batch_size=2,
    report_to="tensorboard"
)


In [ ]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=tokenized,
    eval_dataset=tokenized
)

trainer.train()


In [ ]:
%load_ext tensorboard
%tensorboard --logdir modelo_final

In [ ]:
def chat_psicologia():
    print("=== Chat Psicología (Escribe SALIR para terminar) ===\n")

    while True:
        pregunta = input("Tú: ")

        if pregunta.lower().strip() == "salir":
            print("Fin del chat.")
            break

        prompt = f"""
Actúa como profesor de psicología experto.
Responde únicamente usando tu entrenamiento en los libros.

Pregunta:
{pregunta}

Respuesta:
"""
        inputs = tokenizer(prompt, return_tensors="pt")
        output = model.generate(
            **inputs,
            max_new_tokens=250,
            temperature=0.7
        )

        print("\nModelo:", tokenizer.decode(output[0], skip_special_tokens=True), "\n")

chat_psicologia()
